In [10]:
from embedding_model import Embeddings
from pattern_matching import PatternMatching
from logistic_regression import LR
import pandas as pd
import time
import requests

# Setup

In [14]:
# Embeddings model
targets = {
    1: 'hints_category',
    2: 'hints_category',
    3: 'naics_description',
    4: 'naics_description',
    5: 'hints_tags'
}
business_taxonomy = pd.read_csv('Business_category_taxonomy.csv')
naics_taxonomy = pd.read_csv('Naics3labeltaxonomy.csv')
hints = pd.read_csv('tournament_hints_data.csv')
labels = pd.read_csv('cleaned_naics_codes.csv')
files = {
    'business': business_taxonomy,
    'naics': naics_taxonomy,
    'hints': hints,
    'labels': labels
}

model_name = 'all-MiniLM-L6-v2'
embeddings = Embeddings(model_name, targets, files)

# Pattern matching
pattern_matching = PatternMatching(files['labels'])

# Logistic Regression
lr = LR('pipeline.joblib')

# BERT

# Running cell

In [18]:
base_url = 'http://116.202.111.229:8000'
api_key = 'AjBzyzk6Wg8V6hD48EH8btOOOJOAtYn1'

headers = {
    'x-api-key': api_key
}

# 5 rounds
for i in range(5):
    # Get a new hint for current company or get the first hint for a new company after calling /evaluate/reset
    response = requests.get(f"{base_url}/evaluate/hint", headers=headers)
    print(response.status_code, response.json())

    hint = response.json()['hint']
    level = response.json()['level']
    random_value = naics_taxonomy['naics_code'].sample(1).iloc[0]

    if level == 1:
        pred = pattern_matching.predict_naics(hint)
        if pred is not None:
            pred = lr.predict(hint)
            # pred = embeddings(hint, level)
    elif level == 2:
        pred = embeddings(hint, level)
    elif level == 3:
        pred = embeddings(hint, level)
    elif level == 4:
        pred = embeddings(hint, level)
    elif level == 5:
        pred = embeddings(hint, level)
        
    data = {
        'answer': str(pred)
    }
    response = requests.post(f"{base_url}/evaluate/answer", json=data, headers=headers)
    print(response.status_code, response.json())
    time.sleep(1.5)

# Get hints about a new company
response = requests.get(f"{base_url}/evaluate/reset", headers=headers)
print(response.status_code, response.json())

200 {'company_id': 148, 'level': 2, 'hint': 'Drywall Repair Services | Cabinet Refinishing and Repainting Services | Fully Licensed and Insured | Touch-up Services | Residential and Commercial Painting'}
429 {'title': 'Reached allowed limit 1 hits per 1 second!'}
200 {'company_id': 148, 'level': 3, 'hint': 'Springfield Painting Services is a painter that specializes in residential and commercial painting in the Springfield, MO area.'}
429 {'title': 'Reached allowed limit 1 hits per 1 second!'}
200 {'company_id': 148, 'level': 4, 'hint': 'Springfield Painting Services is a professional painting company that specializes in residential and commercial painting in the Springfield, MO area. They offer a variety of services, including exterior and interior painting, drywall repair, and cabinet refinishing and repainting. Their experienced painters take pride in their work and use high-quality paints and techniques to ensure customer satisfaction. They also provide excellent customer service a

AttributeError: 'NoneType' object has no attribute 'split'

In [17]:
# Get hints about a new company
response = requests.get(f"{base_url}/evaluate/reset", headers=headers)
print(response.status_code, response.json())

200 {'response': 'Wait for 113.16682314872742 seconds'}
